[Link to tutorial](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/learning-networks/)

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Repos/mike_scratch/mlj_tutorial/A-learning-networks`


┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.7.1). Unexpected behavior may occur.
└ @ nothing /Users/mph/Repos/mike_scratch/mlj_tutorial/A-learning-networks/Manifest.toml:0


   Installed MLJMultivariateStatsInterface ─ v0.2.2


Precompiling project...


  ✓ Arpack_jll


  ✓ Arpack


  ✓ MultivariateStats


  ✓ MLJMultivariateStatsInterface
  4 dependencies successfully precompiled in 2 seconds. 88 already precompiled.


In [2]:
using MLJ, StableRNGs
import DataFrames

In [3]:
Ridge = @load RidgeRegressor pkg=MultivariateStats

rng = StableRNG(71)

x1 = rand(rng, 300)
x2 = rand(rng, 300)
x3 = rand(rng, 300)
y = exp.(x1 - x2 - 2*x3 + 0.1 * rand(rng, 300))

X = DataFrames.DataFrame(x1=x1, x2=x2, x3=x3)
first(X,3) |> pretty

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/mph/.julia/packages/MLJModels/tMgLW/src/loading.jl:168


import MLJMultivariateStatsInterface ✔


┌────────────┬────────────┬────────────┐
│ x1         │ x2         │ x3         │
│ Float64    │ Float64    │ Float64    │
│ Continuous │ Continuous │ Continuous │
├────────────┼────────────┼────────────┤
│ 0.894278   │ 0.480202   │ 0.133585   │
│ 0.686442   │ 0.755178   │ 0.0831463  │
│ 0.492613   │ 0.612385   │ 0.923971   │
└────────────┴────────────┴────────────┘


In [4]:
test, train = partition(eachindex(y), 0.8);

A learning network is a DAG whose nodes apply trained or untrained operations, such as predict, transform, `+`, vcat, etc. Here's a fairly standard regression workflow: teh data is standardized, the target is transformed using Box-Cox, a ridge regression is applied and the result is converted back by inverting the transformation.

![A learning network flow](https://juliaai.github.io/DataScienceTutorials.jl/assets/diagrams/composite1.svg)

This DAG is simple enough that it could be done in a pipeline.

## Sources and nodes

In [5]:
Xs = source(X)
ys = source(y)

Source @639 ⏎ `AbstractVector{Continuous}`

In [6]:
stand = machine(Standardizer(), Xs)
W = transform(stand, Xs)

Node{Machine{Standardizer,…}}
  args:
    1:	Source @556
  formula:
    transform(
        Machine{Standardizer,…}, 
        Source @556)

In [7]:
fit!(W, rows=train);

┌ Info: Training Machine{Standardizer,…}.
└ @ MLJBase /Users/mph/.julia/packages/MLJBase/MuLnJ/src/machines.jl:464


Once `W` is fit, we can choose which part of the data to apply it on.

In [9]:
W()               # transform all data.
W(rows=test, )    # transforms only the test data.
W(X[3:4, :])      # transforms only specific data.

,x1,x2,x3
,Float64,Float64,Float64
1,-0.00299567,0.188861,1.34562
2,0.360153,0.56441,1.50152


Now define other nodes.

In [10]:
box_model = UnivariateBoxCoxTransformer()
box = machine(box_model, ys)
z = transform(box, ys)

ridge_model = Ridge(lambda=0.1)
ridge = machine(ridge_model, W, z)
ẑ = predict(ridge, W)

ŷ = inverse_transform(box, ẑ)

Node{Machine{UnivariateBoxCoxTransformer,…}}
  args:
    1:	Node{Machine{RidgeRegressor,…}}
  formula:
    inverse_transform(
        Machine{UnivariateBoxCoxTransformer,…}, 
        predict(
            Machine{RidgeRegressor,…}, 
            transform(
                Machine{Standardizer,…}, 
                Source @556)))

Above we defined the network. Now we can fit the model.

In [11]:
fit!(ŷ, rows=train)

┌ Info: Training Machine{UnivariateBoxCoxTransformer,…}.
└ @ MLJBase /Users/mph/.julia/packages/MLJBase/MuLnJ/src/machines.jl:464
┌ Info: Not retraining Machine{Standardizer,…}. Use `force=true` to force.
└ @ MLJBase /Users/mph/.julia/packages/MLJBase/MuLnJ/src/machines.jl:467


┌ Info: Training Machine{RidgeRegressor,…}.
└ @ MLJBase /Users/mph/.julia/packages/MLJBase/MuLnJ/src/machines.jl:464


Node{Machine{UnivariateBoxCoxTransformer,…}}
  args:
    1:	Node{Machine{RidgeRegressor,…}}
  formula:
    inverse_transform(
        Machine{UnivariateBoxCoxTransformer,…}, 
        predict(
            Machine{RidgeRegressor,…}, 
            transform(
                Machine{Standardizer,…}, 
                Source @556)))

Now that `y\hat` has been fit, we can apply the full graph on the test data. Let's get the `rms` between ground thruth and the predicted values.

In [12]:
rms(y[test], ŷ(rows=test))

0.16668840048398906

## Modifying hyperparameters

Hyperparameters can be accessed using dot syntax as usual. Let's change the `lambda` to 5.0.

In [13]:
ridge_model.lambda = 5.0;

Since the node `z\hat` corresponds to a machine that wraps `ridge_model`, that node has effectively changed and will be retrained.

In [14]:
fit!(ŷ, rows=train)
rms(y[test], ŷ(rows=test))

┌ Info: Not retraining Machine{UnivariateBoxCoxTransformer,…}. Use `force=true` to force.
└ @ MLJBase /Users/mph/.julia/packages/MLJBase/MuLnJ/src/machines.jl:467
┌ Info: Not retraining Machine{Standardizer,…}. Use `force=true` to force.
└ @ MLJBase /Users/mph/.julia/packages/MLJBase/MuLnJ/src/machines.jl:467
┌ Info: Updating Machine{RidgeRegressor,…}.
└ @ MLJBase /Users/mph/.julia/packages/MLJBase/MuLnJ/src/machines.jl:465


0.08107794068129705